In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
import random
import csv

personality_data = pd.read_csv('./data/PersonalityData.csv', names=["user_id", "openness", "agreeableness", "emotional_stability", "conscientiousness", "extraversion"])
print("pdf", personality_data.shape)
rating_data = pd.read_csv('./data/Rating.csv', names=["user_id", "movie_id", "rating", "timestamp"])
rating_data = rating_data.drop("timestamp", axis=1)
tag_data = pd.read_csv('./data/MovieTag.csv', names=["movie_id", "tag"])

print("rdf", rating_data.shape)
whole_data = pd.merge(personality_data, rating_data, left_on='user_id', right_on='user_id', how='inner')
whole_data = pd.merge(whole_data, tag_data, left_on = 'movie_id', right_on='movie_id', how='inner')
print(whole_data.head())

pdf (1820, 6)
rdf (912643, 3)
                            user_id  openness  agreeableness  \
0  8e7cebf9a234c064b75016249f2ac65e       5.0            2.0   
1  8e7cebf9a234c064b75016249f2ac65e       5.0            2.0   
2  b7e8a92987a530cc368719a0e60e26a3       4.0            3.0   
3  b7e8a92987a530cc368719a0e60e26a3       4.0            3.0   
4  92561f21446e017dd6b68b94b23ad5b7       5.5            5.5   

   emotional_stability  conscientiousness  extraversion  movie_id  rating  \
0                  3.0                2.5           6.5         1     5.0   
1                  3.0                2.5           6.5         1     5.0   
2                  4.5                2.0           2.5         1     3.5   
3                  4.5                2.0           2.5         1     3.5   
4                  4.0                4.5           4.0         1     2.0   

     tag  
0  pixar  
1    fun  
2  pixar  
3    fun  
4  pixar  


In [2]:
tags = list(set(whole_data['tag']))
tags.remove(tags[0])

def specificMovieData(table, tag):
    return table.loc[table['tag'] == tag]

aggregate_personality = []
# tag_id_to_tag = []

# tag_id = 0

for idx in range(len(tags)):
    tag = tags[idx]
    tag_data = specificMovieData(whole_data,tag)
    tag_data = tag_data.drop('user_id', axis=1)
    tag_data = tag_data.drop('movie_id', axis=1)
    
    tag_data["openness"] = tag_data["openness"] * tag_data["rating"]
    tag_data["agreeableness"] = tag_data["agreeableness"] * tag_data["rating"]
    tag_data["emotional_stability"] = tag_data["emotional_stability"] * tag_data["rating"]
    tag_data["conscientiousness"] = tag_data["conscientiousness"] * tag_data["rating"]
    tag_data["extraversion"] = tag_data["extraversion"] * tag_data["rating"]
    
    openness = tag_data["openness"].sum()
    agreeableness = tag_data["agreeableness"].sum()
    emotional_stability = tag_data["emotional_stability"].sum()
    conscientiousness = tag_data["conscientiousness"].sum()
    extraversion = tag_data["extraversion"].sum()
    
    tag_personality = [tag, openness, agreeableness, emotional_stability, conscientiousness, extraversion]
    aggregate_personality.append(tag_personality)
#     tag_id_to_tag.append([tag_id, tag])
    
#     tag_id += 1

#movie_id tag openness agreeableness emotional_stability conscientiousness extraversion
    

In [3]:
with open('./data/TagPersonalities.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(aggregate_personality)
    
# with open('./data/Tags.csv', 'w') as f:
#     w = csv.writer(f)
#     w.writerows(tag_id_to_tag)
    